In [1]:
%load_ext autoreload
%autoreload 2

import os
import sys
sys.path.append("../../src")
import pandas as pd
import numpy as np
import synapseclient as sc
import utils.munging_utils as query

Welcome, aryton tediarjo!



In [2]:
df = query.get_walking_synapse_table(recordIds = ["db3d35fe-4798-462d-ab01-eb7bb47b75ff"],
                               table_id = "syn12514611",
                               version = "V2")

Querying V2 Data


In [3]:
def get_fs(path):
    data = query.open_filepath(path)[1:]
    data = data[data["sensorType"] == "acceleration"]
    time_end = (data["timestamp"][-1:].iloc[0])
    N = data.shape[0]
    return N/time_end

In [4]:
data = query.open_filepath(df["walk_motion.json_pathfile"].iloc[0])

In [8]:
data[data["sensorType"] == "rotationRate"]

,eventAccuracy,referenceCoordinate,sensorType,stepPath,timestamp,timestampDate,uptime,w,x,y,z
18,NaN,NaN,rotationRate,WalkAndBalance/walk/motion,0.172541,NaN,877289.877609,NaN,-0.001646,-0.001787,-0.002233
25,NaN,NaN,rotationRate,WalkAndBalance/walk/motion,0.182545,NaN,877289.887613,NaN,0.001685,0.001361,-0.009638
30,NaN,NaN,rotationRate,WalkAndBalance/walk/motion,0.192545,NaN,877289.897613,NaN,0.000571,0.003500,-0.006455
37,NaN,NaN,rotationRate,WalkAndBalance/walk/motion,0.202550,NaN,877289.907618,NaN,-0.001745,0.001426,0.004156
43,NaN,NaN,rotationRate,WalkAndBalance/walk/motion,0.212555,NaN,877289.917623,NaN,-0.000809,-0.002783,0.010545
48,NaN,NaN,rotationRate,WalkAndBalance/walk/motion,0.222555,NaN,877289.927623,NaN,0.000357,0.000373,0.005243
55,NaN,NaN,rotationRate,WalkAndBalance/walk/motion,0.232560,NaN,877289.937628,NaN,0.000590,0.003486,-0.007519
61,NaN,NaN,rotationRate,WalkAndBalance/walk/motion,0.242564,NaN,877289.947632,NaN,-0.001499,0.001339,-0.009676
68,NaN,NaN,rotationRate,WalkAndBalance/walk/motion,0.252564,NaN,877289.957632,NaN,-0.000567,0.001375,-0.002216
73,NaN,NaN,rotationRate,WalkAndBalance/walk/motion,0.262556,NaN,877289.967624,NaN,0.000278,-0.001739,0.009495
